# Urban Henges with OSMNX

This was inspired by @Puntofisso's work colouring streetnames using OSMnx.

I wanted to use python and OMNX to create animations to show urban 'henges' - like ManhattanHenge. This code highlights where urban streets align with the angle of the rising or setting sun throughout the year. At the summer solstice, the sun rises in the North East; at the winter solstice it's close to the South East. By iterating through the possible angles, I created frames to assemble into a GIF. Streets are highlighted at the angle of the rising sun.

For an urban henge to really work, you would need a view of the horizon - quite challenging in an urban context. So these 'henges' are theoretical. You would also see the full effect of the sun when it's a couple of degrees above the horizon, so I've designed the highlight to start at the angle of the rising sun and grow for a couple of degrees before decaying.

OSMnx is an amazing package which allows you to download Open Streetmaps as networks of streets to manipulate in python with Networkx and Pandas. It's created and maintained by Geoff Boeing @gboeing
https://geoffboeing.com/2016/11/osmnx-python-street-networks/


In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import math
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os
from datetime import datetime, timedelta
%matplotlib inline
ox.config(log_console=True, use_cache=True)



## Functions

In [2]:


def make_graph(place, which_result):
    # makes the graph structure; depending on the size/administrative level of the city, 
    # may need to change the 'which result' variable to 1 or 2
    G = ox.graph_from_place(place, network_type='all', which_result=which_result)
    return G
    
def make_df(G):
    # turns the graph into a df of streets
    df = ox.graph_to_gdfs(G, nodes=False)
    return df
    


def find_angle(street):
    # finds the angle of a street given two coordinate points given in the graph network
    if len(street.xy[0]) > 2:
        return 0
    else:
        X1, X2 = street.xy[0][0], street.xy[0][1]
        Y1, Y2 = street.xy[1][0], street.xy[1][1]
        x = X1 - X2
        y = Y1 - Y2
        if x == 0:
            return 0
        if y == 0:
            return 0
        else:
            return round(math.degrees(math.atan(x/y)), 1)


def fade(value):
    # function for making a fade effect for the width of the street. This uses a gaussian
    # curve to make the linewidth 'decay' and fade away
    mean = 2; std = 0.5; variance = np.square(std)
    return 6*(np.exp(-np.square(value-mean)/2*variance)/(np.sqrt(2*np.pi*variance)))


def colour_fading(angle, solstice):
    # applies the fade effect depending on whether the street is at the exact angle of
    # the rising/setting sun
    a = solstice - angle
    b = solstice - angle - 180 
    
    if -6 < a < 6:
        return {'colour':'red', 'linewidth':fade(a), 'alpha':1.0}
    
    if -6 < b < 6:
        return {'colour':'red', 'linewidth':fade(b), 'alpha':1.0}

    else:
        #colour of the non-highlighted streets
        return {'colour':'grey', 'linewidth':0.7, 'alpha':1} 

    
def add_features(df, solstice):
    # adds new features to the df to help with customising the plot: the angle of the street,
    # the weight and colour of the street. Note matplotlib won't take a series for the alpha
    # value so it's not actually possible to customise this
    df['angle'] = df['geometry'].apply(lambda x: find_angle(x))
    df['colour'] = df['angle'].apply(lambda x: colour_fading(x, solstice)['colour'])
    df['linewidth'] = df['angle'].apply(lambda x: colour_fading(x, solstice)['linewidth'])
    df['alpha'] = df['angle'].apply(lambda x: colour_fading(x, solstice)['alpha'])
    
    return df    


def make_map(G, df, filename, figsize):
    # draws the map, with option to customise the figsize                    
    fig, ax = ox.plot_graph(G, bgcolor='black', axis_off=True, node_size=0, node_color='grey', node_edgecolor='grey', node_zorder=2,
                        edge_color=df['colour'], edge_linewidth=df['linewidth'], edge_alpha=.7, fig_height=figsize, dpi=50)
       
    return fig.savefig(filename+'.png', facecolor=fig.get_facecolor())


def make_animation(Graph, save_as, figsize, version):
    # creates directories to save the images in and then assembles them into a GIF. Option to
    # try different sizes for a bigger or smaller final size
    
    # create directory
    main_dir = ('main_dir')
    png_dir = 'images'
    gif_dir = 'GIFs'
    os.chdir(main_dir) # enter the full path
    if not os.path.exists(main_dir + '/' + save_as):
        os.mkdir(main_dir + '/' + save_as)
    if not os.path.exists(main_dir + save_as + '/' + png_dir):
        os.mkdir(main_dir + save_as + '/' + png_dir)
    if not os.path.exists(main_dir + save_as + '/' + gif_dir):
        os.mkdir(main_dir + save_as + '/' + gif_dir)

    
    # make maps
    os.chdir('main_dir'+ save_as+'/images')

    master_df = make_df(Graph)
    for solstice in range (49, 129):
        df = add_features(master_df, solstice)
        make_map(Graph, df, save_as+str(solstice), figsize)

    # make GIF
    
    os.chdir('main_dir'+ save_as)

    # Create a list of filenames going forwards and backwards
    f = [save_as+str(x)+'.png' for x in range(49, 129)]
    filenames = f + f[::-1]

    # Create the frames
    images=[]
    for file_name in filenames:
        file_path = os.path.join(png_dir, file_name)
        #print(file_path)
        file = Image.open(file_path)

        images.append(file)
        file.load()

    # Save into a GIF file that loops forever

    images[0].save(gif_dir + '/' + save_as + str(version) + '.gif', format='GIF',
                   append_images=images[1:],
                   save_all=True, loop=0, duration=100, optimise=True)
    
    return



## San Francisco

In [201]:
SanFran = make_graph('San Francisco, USA', 1)


In [ ]:
make_animation(SanFran, 'SanFran', 7, version=1)